In [2]:
%pip install gensim nltk transformers torch annoy seaborn matplotlib scikit-learn PyPDF2 plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 3.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 243.6 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import PyPDF2
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import pickle

def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = " ".join([page.extract_text() for page in pdf_reader.pages if page.extract_text() is not None])
    return text

def preprocess_text(text):
    # Tokenizes and processes the text. 
    # Modify as needed (e.g., lowercasing, removing stopwords)
    return word_tokenize(text.lower())

def process_pdf(file_path):
    try:
        document_text = read_pdf(file_path)
        processed_text = preprocess_text(document_text)
        return processed_text, file_path
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return [], file_path

def build_doc2vec_model(folder_path, num_papers=None, vector_size=100, window=5, min_count=2, workers=4):
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.pdf')]
    processed_files = all_files[:num_papers] if num_papers else all_files

    tagged_data = []
    for file_path in tqdm(processed_files):
        words, path = process_pdf(file_path)
        tagged_data.append(TaggedDocument(words=words, tags=[path]))

    model = Doc2Vec(vector_size=vector_size, window=window, min_count=min_count, workers=workers)
    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

    return model

def create_doc_embeddings(folder_path, num_papers=None):
    model = build_doc2vec_model(folder_path, num_papers)
    embeddings_dict = {doc.tags[0]: model.dv[doc.tags[0]] for doc in model.dv.vectors_docs}
    return embeddings_dict

# Set the path to the directory containing the papers
papers_path = './PapersDirectory/papers'

# Create embeddings
embeddings_dict = create_doc_embeddings(papers_path)
# Save the embeddings dictionary
with open('doc_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings_dict, f)


  8%|▊         | 123/1464 [01:17<08:35,  2.60it/s]FloatObject (b'0.00-29537734') invalid; use 0.0 instead
FloatObject (b'0.00-29537734') invalid; use 0.0 instead
FloatObject (b'0.00-29537734') invalid; use 0.0 instead
FloatObject (b'0.00-29537734') invalid; use 0.0 instead
 12%|█▏        | 174/1464 [06:00<09:25,  2.28it/s]   FloatObject (b'0.000000000-2217712') invalid; use 0.0 instead
FloatObject (b'0.000000000-4435424') invalid; use 0.0 instead
FloatObject (b'0.000000000-2217712') invalid; use 0.0 instead
FloatObject (b'0.000000000-4435424') invalid; use 0.0 instead
 22%|██▏       | 324/1464 [11:46<06:46,  2.80it/s]   FloatObject (b'0.00-6711408') invalid; use 0.0 instead
FloatObject (b'0.00-6711408') invalid; use 0.0 instead
 36%|███▋      | 531/1464 [14:21<09:13,  1.69it/s] impossible to decode XFormObject /F1-DejaVuSans-Idotaccent
 impossible to decode XFormObject /F1-DejaVuSans-Idotaccent
 42%|████▏     | 617/1464 [15:32<09:58,  1.41it/s]/opt/homebrew/lib/python3.11/site-packages